In [1]:
! pip install git+https://github.com/openai/whisper.git -q
! pip install gradio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta

In [2]:
!pip install gtts

In [3]:
import whisper
import gradio as gr
from gtts import gTTS
import os

# Load Whisper Model
model = whisper.load_model("medium")

# Supported Languages
LANGUAGES = {
    "auto": "Auto Detect",
    "en": "English",
    "tl": "Tagalog"
}
LANGUAGE_CODES = {v: k for k, v in LANGUAGES.items()}

def transcribe_and_tts(audio, selected_language_name):
    try:
        # Convert language name to code
        selected_language = LANGUAGE_CODES[selected_language_name]

        # Load and preprocess audio
        audio = whisper.load_audio(audio)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # Detect language if auto is selected
        if selected_language == "auto":
            _, probs = model.detect_language(mel)
            detected_lang = max(probs, key=probs.get)
            print(f"Detected language: {detected_lang}")
            lang_to_use = detected_lang
        else:
            lang_to_use = selected_language

        # Transcribe the audio
        result = model.transcribe(
            audio,
            language=lang_to_use if selected_language != "auto" else None,
            temperature=0.0,
            beam_size=5
        )
        text_output = result["text"]

        # Convert text to speech using gTTS
        tts = gTTS(text=text_output, lang=lang_to_use if lang_to_use in ["en", "tl"] else "en")
        tts_file = "output.mp3"
        tts.save(tts_file)

        return text_output, tts_file

    except Exception as e:
        return f"Error: {str(e)}", None

# Create Gradio Interface
demo = gr.Interface(
    fn=transcribe_and_tts,
    inputs=[
        gr.Audio(sources=["microphone", "upload"], type="filepath", label="Audio Input"),
        gr.Dropdown(choices=list(LANGUAGES.values()), value="Auto Detect", label="Select Language")
    ],
    outputs=[
        gr.Textbox(label="Transcription"),
        gr.Audio(label="Generated Speech")
    ],
    title="Whisper Speech-to-Speech Demo",
    description="Upload audio or record from microphone. Get transcription and synthesized speech output."
)

demo.launch()


100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 91.6MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2eee05befff85842d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
